# Módulo 1: Web Scraping de Detalhes dos Anúncios do Airbnb

## 1. O Ponto de Partida: Um Dataset Incompleto

No estágio anterior de nosso projeto, executamos um script de extração (`2 - exemplo_script_extracao_anuncios.py`) que varreu as páginas de busca do Airbnb, coletando informações gerais sobre milhares de anúncios, como ID, título, preço por período e o link para a página principal.

O resultado foi um arquivo CSV robusto, porém, incompleto. Para uma análise mais profunda e para popular nosso banco de dados corretamente, precisamos de detalhes que só existem *dentro* da página de cada anúncio.

### O Desafio
Como podemos, de forma automática e eficiente, visitar milhares de links para extrair informações específicas como o número de quartos, camas, banheiros e as regras de check-in/checkout? Fazer isso manualmente seria impossível.

### A Solução: Um Robô de Extração
Este notebook é a solução. Ele atua como um "robô" (web scraper) que foi programado para realizar as seguintes tarefas:
1.  Ler a lista de anúncios do nosso arquivo CSV.
2.  Visitar, um por um, o link de cada anúncio único.
3.  "Ler" a página e extrair de forma inteligente os detalhes que faltam.
4.  Salvar esses novos dados de forma estruturada para uso futuro.

Vamos começar!

## 2. Preparando o Ambiente: As Ferramentas Necessárias

Antes de construirmos nosso robô, precisamos importar nossa "caixa de ferramentas" de bibliotecas Python. Cada uma tem um papel fundamental:

* **Pandas**: Para ler e organizar nossos dados em tabelas (DataFrames).
* **Selenium**: A biblioteca principal que nos permite controlar um navegador de internet (Firefox, neste caso) através de código.
* **BeautifulSoup**: Uma ferramenta fantástica para "ler" o HTML de uma página web e encontrar as informações que queremos de forma fácil.
* **OS, Sys, Time, Re**: Bibliotecas auxiliares para interagir com arquivos e pastas, controlar o tempo e encontrar padrões em textos.

In [ ]:
# --- Bibliotecas para Manipulação de Dados e Arquivos ---
import pandas as pd  # Usado para criar e manipular DataFrames (nossas tabelas de dados).
import os            # Usado para interagir com o sistema operacional, como verificar se um arquivo existe.
import sys           # Permite interagir com o sistema, usado aqui para parar o script em caso de erro.
import re            # Biblioteca de expressões regulares, para encontrar padrões em textos.
import time          # Usado para adicionar pausas (esperas) no código, essenciais em web scraping.

# --- Bibliotecas para Web Scraping e Automação ---
from selenium import webdriver  # A ferramenta principal que controla o navegador.
from selenium.webdriver.firefox.options import Options  # Permite configurar as opções do navegador (ex: modo headless).
from selenium.webdriver.common.by import By  # Usado para especificar como encontrar elementos na página (por ID, XPath, etc.).
from selenium.webdriver.support.ui import WebDriverWait  # Permite criar esperas inteligentes, aguardando que uma condição aconteça.
from selenium.webdriver.support import expected_conditions as EC  # Contém as condições para as esperas inteligentes (ex: elemento visível).
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # Classes de erro específicas do Selenium para lidar com imprevistos.
from bs4 import BeautifulSoup  # Biblioteca para analisar (parse) o código HTML da página e facilitar a extração de dados.

## 3. Construindo o Coração do Robô: A Função de Extração

Agora, vamos ao código mais importante deste notebook. A função `extrair_detalhes_anuncio` é o "cérebro" do nosso robô. É aqui que ensinamos a ele exatamente como se comportar ao visitar uma página do Airbnb.

Pense nesta função como o "manual de instruções" do robô para uma única tarefa: ao receber um link, ele deve saber exatamente onde procurar por "quartos", "camas", "banheiros" e as regras da casa. Como as páginas podem mudar, programamos múltiplos métodos de busca (um principal e um alternativo) para torná-lo mais resiliente a erros.

In [ ]:
def extrair_detalhes_anuncio(driver, url):
    """
    Navega para a URL de um anúncio e extrai detalhes como número de quartos,
    camas, banheiros e horários de check-in/check-out.
    """
    # Limpa a URL para garantir que estamos acessando a página principal do anúncio, removendo parâmetros de busca.
    base_url = url.split('?')[0].split('/house-rules')[0]

    # Inicializa as variáveis como 'None'. Isso garante que, se uma extração falhar, o valor será nulo e não um erro.
    quartos = None
    camas = None
    banheiros = None
    horario_checkin = None
    horario_checkout = None

    # --- Bloco 1: Extração de detalhes da acomodação (Quartos, Camas, Banheiros) ---
    print("  - Extraindo Quartos, Camas e Banheiros...")
    try:
        # Define um tempo máximo de 20 segundos para a página carregar. Se demorar mais, lança um erro (TimeoutException).
        driver.set_page_load_timeout(20)
        # O navegador acessa a URL do anúncio.
        driver.get(base_url)
        # Aguarda de forma inteligente (por até 20s) que a tag <body> da página esteja presente, indicando que o HTML básico foi carregado.
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        # Adiciona uma pausa estática de 3 segundos para garantir que scripts dinâmicos (Javascript) tenham tempo de renderizar o conteúdo.
        time.sleep(3) 

        # Pega o código-fonte HTML completo da página após o carregamento e o entrega ao BeautifulSoup para análise.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Tentativa 1 (Método Principal): Busca a seção de "visão geral" que geralmente contém os detalhes.
        overview_section = soup.find('div', {'data-plugin-in-point-id': 'OVERVIEW_DEFAULT_V2'})
        if overview_section:
            # Se a seção for encontrada, busca todos os itens de lista ('li') dentro dela.
            lista_itens = overview_section.find_all('li', class_='l7n4lsf')
            # Itera sobre cada item encontrado.
            for item in lista_itens:
                # Limpa o texto do item, removendo caracteres como '·' e espaços em branco.
                texto_item = item.get_text(strip=True).replace('·', '').strip()
                # Verifica se o texto contém palavras-chave e atribui à variável correta.
                if 'quarto' in texto_item:
                    quartos = texto_item
                elif 'cama' in texto_item:
                    camas = texto_item
                elif 'banheiro' in texto_item:
                    banheiros = texto_item
        else:
            # Tentativa 2 (Método Alternativo): Se o método principal falhar (pois o site pode mudar), este é executado.
            print("    Seção de visão geral não encontrada. Tentando método alternativo.")
            try:
                # Localiza um texto de referência ("hóspedes") para encontrar a seção correta.
                overview_element = driver.find_element(By.XPATH, "//*[contains(text(), 'hóspedes')]")
                # A partir do texto, navega para o elemento "pai" que contém todos os detalhes.
                parent_div = overview_element.find_element(By.XPATH, "./..")
                # Pega todos os textos dentro do elemento pai.
                items = parent_div.find_elements(By.TAG_NAME, 'span')
                # Junta todos os pedaços de texto em uma única string.
                full_text = ' '.join([item.text for item in items if item.text.strip()])

                # Usa expressões regulares (regex) para extrair os padrões exatos que queremos.
                q = re.search(r'(\d+\s*quarto|Estúdio)', full_text)
                c = re.search(r'(\d+\s*cama)', full_text)
                b = re.search(r'(\d+\s*banheiro)', full_text)
                # Se um padrão for encontrado (match), armazena o resultado.
                if q: quartos = q.group(1)
                if c: camas = c.group(1)
                if b: banheiros = b.group(1)
            except Exception:
                # Se o método alternativo também falhar, ele apenas ignora o erro e continua.
                pass 

    except TimeoutException:
        # Este erro é capturado se a página demorar mais que o tempo limite para carregar.
        print("  - A página principal demorou muito para carregar. Pulando extração de quartos/camas/banheiros.")
    except Exception as e:
        # Captura qualquer outro erro inesperado para evitar que o script pare.
        print(f"  ERRO ao extrair quartos/camas/banheiros: {e}")
        # Atribui um valor de 'Erro' para sabermos que a extração falhou para este item.
        quartos, camas, banheiros = 'Erro', 'Erro', 'Erro'

    # Imprime os resultados encontrados para acompanhamento em tempo real no console.
    print(f"    Quartos: {quartos}, Camas: {camas}, Banheiros: {banheiros}")

    # --- Bloco 2: Extração de Horários de Check-in e Check-out ---
    print("  - Extraindo Check-in/Check-out...")
    try:
        # Cria um novo 'soup' para garantir que temos o estado mais atualizado da página.
        soup_regras = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Tentativa 1: Busca a seção de políticas da casa diretamente.
        policies_section = soup_regras.find('div', {'data-section-id': 'POLICIES_DEFAULT'})
        if policies_section:
            rule_items = policies_section.find_all('div', class_='i1303y2k')
            for item in rule_items:
                texto_item = item.get_text(strip=True)
                if 'Check-in' in texto_item:
                    horario_checkin = texto_item
                elif 'Checkout' in texto_item:
                    horario_checkout = texto_item

        # Tentativa 2 (Fallback): Se as regras não estavam visíveis, tenta clicar no botão "Mostrar mais".
        if not horario_checkin or not horario_checkout:
            print("    Check-in/Check-out não encontradas, tentando clicar em 'Mostrar mais'...")
            try:
                # Aguarda por até 5 segundos que o botão se torne clicável.
                show_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Mostrar regras da casa')] | //button[contains(., 'Mostrar mais')]"))
                )
                # Clica no botão usando JavaScript, uma técnica mais robusta.
                driver.execute_script("arguments[0].click();", show_more_button)
                # Espera 2 segundos para a janela (modal) com as regras abrir.
                time.sleep(2)

                # Analisa o novo código HTML da página, agora com o modal aberto.
                soup_modal = BeautifulSoup(driver.page_source, 'html.parser')
                modal_rule_items = soup_modal.find_all('div', class_='f15dgkuj')
                for item in modal_rule_items:
                    texto_item = item.get_text(strip=True)
                    if 'Check-in:' in texto_item:
                        horario_checkin = texto_item
                    elif 'Checkout:' in texto_item:
                        horario_checkout = texto_item
            except (TimeoutException, NoSuchElementException):
                # Erro comum se o botão não existir ou não aparecer a tempo.
                print("    Botão 'Mostrar mais' para Check-in/Check-out não encontrado.")
            except Exception as e:
                # Captura de outros erros possíveis ao interagir com o modal.
                print(f"    Erro ao tentar abrir o modal de Check-in/Check-out: {e}")

    except Exception as e:
        # Captura de erro geral para este bloco.
        print(f"  ERRO FATAL ao extrair os horários de check-in/out: {e}")
        horario_checkin, horario_checkout = 'Erro na extração', 'Erro na extração'
    finally:
        # Garante que o tempo de espera do driver seja restaurado para o padrão (60s), evitando problemas na próxima iteração.
        driver.set_page_load_timeout(60)

    # Imprime os resultados para acompanhamento.
    print(f"    Check-in: {horario_checkin}")
    print(f"    Check-out: {horario_checkout}")

    # Retorna um dicionário com todos os dados coletados.
    return {
        "Quartos": quartos,
        "Camas": camas,
        "Banheiros": banheiros,
        "Horário de Check-in": horario_checkin,
        "Horário de Check-out": horario_checkout,
    }

## 4. Colocando o Robô para Trabalhar: O Processo de Extração

Com o "cérebro" do nosso robô (`extrair_detalhes_anuncio`) devidamente construído, é hora de orquestrar a operação completa. A seção a seguir irá:
1.  Definir os arquivos de trabalho.
2.  Carregar a lista de "tarefas" (os links a serem visitados).
3.  Ligar o navegador.
4.  Executar o loop que passará por cada tarefa, uma por uma.

### 4.1. Definindo o Alvo e os Arquivos de Saída

O primeiro passo é dizer ao robô qual arquivo contém a lista de links para visitar. A partir desse nome, ele irá gerar automaticamente os nomes para o arquivo de resultados parciais (o "backup" do progresso) e o arquivo de saída final.

### 4.2. Carregamento e Preparação dos Dados de Entrada
Lemos o arquivo de entrada e preparamos a lista de imóveis únicos a serem processados. Também verificamos se já existe um arquivo de resultados parciais para continuar o trabalho de onde parou.

In [ ]:
# --- CONFIGURAÇÃO MANUAL ---
# ATENÇÃO: Altere a linha abaixo para indicar o caminho e nome do seu arquivo CSV de entrada.
input_filename = 'dados/airbnb_dados_gerais_1_hospede_4_noites2025_07_08.csv'

# --- GERAÇÃO AUTOMÁTICA DOS NOMES DE SAÍDA ---
# Pega o nome do arquivo de entrada sem a extensão .csv.
base_name, extension = os.path.splitext(input_filename)
# Cria o nome do arquivo para salvar os resultados parciais (ex: 'meus_dados_parciais.csv').
partial_results_filename = f"{base_name}_resultados_parciais.csv"
# Cria o nome do arquivo final que conterá todos os dados combinados.
final_output_filename = f"{base_name}_completo.csv"

# Imprime os nomes dos arquivos para verificação.
print(f"Arquivo de Entrada: {input_filename}")
print(f"Arquivo de Backup (Resultados Parciais): {partial_results_filename}")
print(f"Arquivo de Saída Final: {final_output_filename}")

### 4.2. Carregando a Lista de Tarefas e Verificando Trabalhos Anteriores

Agora, o código lê o arquivo de entrada. Mas, antes de começar o trabalho pesado, ele faz uma verificação inteligente: procura pelo arquivo de resultados parciais. 

Se ele existir, o robô é inteligente o suficiente para ler o que já foi feito e pular os anúncios que já visitou, economizando um tempo valioso e permitindo que o trabalho continue de onde parou.

In [ ]:
# Tenta carregar o arquivo CSV de entrada.
try:
    df_airbnb = pd.read_csv(input_filename)
    print(f"\nArquivo '{input_filename}' carregado com sucesso. Total de {len(df_airbnb)} linhas.")
# Se o arquivo não for encontrado, exibe uma mensagem de erro crítica.
except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo de entrada '{input_filename}' não foi encontrado. Verifique o nome e o caminho.")
    # sys.exit(1) # Este comando pararia o script; em um notebook, o ideal é apenas exibir o erro.

# Valida se o DataFrame carregado contém as colunas essenciais para o script funcionar.
if 'ID Imóvel' not in df_airbnb.columns or 'Link' not in df_airbnb.columns:
    print("ERRO CRÍTICO: O arquivo CSV de entrada precisa ter as colunas 'ID Imóvel' e 'Link'.")
    # sys.exit(1) 

# Prepara a lista de tarefas do robô em um novo DataFrame:
# 1. Seleciona apenas as colunas 'ID Imóvel' e 'Link'.
# 2. Remove todas as linhas que são duplicatas com base na coluna 'ID Imóvel', para processar cada anúncio apenas uma vez.
# 3. Remove quaisquer linhas onde a coluna 'Link' seja nula (NaN), pois não seria possível visitá-las.
df_para_scrape = df_airbnb[['ID Imóvel', 'Link']].drop_duplicates(subset=['ID Imóvel']).dropna(subset=['Link'])
total_links_unicos = len(df_para_scrape)
print(f"Encontrados {total_links_unicos} imóveis únicos para processar.")

# Lógica para retomar o trabalho:
# Inicializa um 'set' (conjunto) vazio para armazenar os IDs já processados. Um 'set' é mais rápido que uma lista para verificações.
processed_ids = set() 
# Verifica se o arquivo de backup já existe no diretório.
if os.path.exists(partial_results_filename):
    print(f"Encontrado arquivo de backup de resultados parciais: '{partial_results_filename}'.")
    # Se existe, carrega o arquivo.
    df_parcial = pd.read_csv(partial_results_filename)
    # Garante que o arquivo de backup tenha a coluna 'ID Imóvel' para evitar erros.
    if 'ID Imóvel' in df_parcial.columns:
        # Adiciona todos os IDs do arquivo de backup ao nosso conjunto de IDs já processados.
        processed_ids = set(df_parcial['ID Imóvel'])
        print(f"Retomando trabalho. {len(processed_ids)} imóveis já foram processados e serão pulados.")
    else:
        # Caso o arquivo parcial esteja corrompido ou sem a coluna necessária.
        print("AVISO: O arquivo de resultados parciais não contém a coluna 'ID Imóvel' e será ignorado.")

### 4.3. Ligando os Motores: Inicialização do Navegador

É hora de iniciar o Selenium. Para máxima eficiência, operamos o navegador em "modo fantasma" (headless), ou seja, sem abrir uma janela visual. Também o instruímos a não carregar imagens e folhas de estilo (CSS), tornando a navegação muito mais rápida e focada apenas no que importa: os dados em HTML.

In [ ]:
# Cria um objeto de configuração para o navegador Firefox.
options = Options()
# Adiciona o argumento para rodar o navegador em modo headless (em segundo plano, sem janela visível).
options.add_argument('--headless')
# Desabilita o carregamento de imagens (valor 2) para acelerar o carregamento da página.
options.set_preference("permissions.default.image", 2)
# Desabilita o carregamento de arquivos de estilo (CSS), o que também acelera a navegação.
options.set_preference("permissions.default.stylesheet", 2)
# Desabilita o download de fontes customizadas da web.
options.set_preference("gfx.downloadable_fonts.enabled", False)

# Imprime uma mensagem para o usuário saber que o navegador está sendo iniciado.
print("\nIniciando o navegador Firefox em modo headless...")
# Inicia o driver do Firefox com todas as opções que configuramos. O robô está pronto para a ação.
driver = webdriver.Firefox(options=options)

### 4.4. A Maratona de Extração: O Loop Principal

Esta é a célula onde a mágica acontece. O robô começa sua jornada, passando por cada anúncio da lista, um por um. Ele chama a função que construímos anteriormente (`extrair_detalhes_anuncio`) para cada link, coleta os detalhes e os salva imediatamente no arquivo de backup.

Para garantir que ele não se "canse" (ou consuma muita memória durante longas extrações), nós o programamos para reiniciar a si mesmo a cada 50 visitas.

In [ ]:
# Contador para saber quantos itens foram processados nesta sessão específica do script.
processed_in_this_session = 0
# Calcula o número total de itens que realmente precisam ser processados, subtraindo os que já foram feitos.
total_a_processar = total_links_unicos - len(processed_ids)
print(f"\nIniciando a extração para {total_a_processar} novos anúncios únicos...")

# Itera sobre cada linha do DataFrame que contém a lista de tarefas (links).
for index, row in df_para_scrape.iterrows():
    # Extrai o ID e a URL da linha atual.
    id_imovel = row['ID Imóvel']
    url = row['Link']

    # Lógica de retomada: se o ID do imóvel atual já está no conjunto de IDs processados, pula para o próximo.
    if id_imovel in processed_ids:
        continue # A palavra-chave 'continue' interrompe a iteração atual e vai para a próxima.

    # Lógica de reinicialização: a cada 50 itens, fecha e abre o navegador para limpar a memória RAM.
    # O operador '%' (módulo) retorna o resto de uma divisão. Se o resto de processed_in_this_session / 50 for 0, reinicia.
    if processed_in_this_session > 0 and processed_in_this_session % 50 == 0:
        print(f"\n--- Processados {processed_in_this_session} links nesta sessão. Reiniciando o navegador... ---")
        driver.quit() # Fecha completamente o navegador atual.
        time.sleep(5) # Pausa de 5 segundos para garantir que o processo do navegador foi totalmente encerrado.
        driver = webdriver.Firefox(options=options) # Inicia uma nova instância do navegador com as mesmas configurações.
        print("--- Navegador reiniciado. Continuando a extração... ---\n")

    # Imprime o progresso atual para acompanhamento.
    print(f"Processando anúncio {processed_in_this_session + 1}/{total_a_processar} (ID: {id_imovel})")

    # Antes de chamar a função, verifica se a URL é válida (não é nula, é um texto e começa com http).
    if pd.notna(url) and isinstance(url, str) and url.startswith("http"):
        # Chama a função principal para extrair os detalhes da página, passando o driver e a url.
        detalhes = extrair_detalhes_anuncio(driver, url)
    else:
        # Se a URL for inválida, registra um dicionário de erro para não quebrar o processo.
        print(f"Link inválido ou ausente para o ID Imóvel {id_imovel}. Pulando.")
        detalhes = {
            "Quartos": 'Link Inválido', "Camas": 'Link Inválido',
            "Banheiros": 'Link Inválido', "Horário de Check-in": 'Link Inválido',
            "Horário de Check-out": 'Link Inválido'
        }

    # Adiciona o ID do imóvel ao dicionário de resultados para garantir a correspondência correta na hora de salvar.
    detalhes['ID Imóvel'] = id_imovel
    # Converte o dicionário de resultado em um pequeno DataFrame de uma linha.
    df_resultado_atual = pd.DataFrame([detalhes])

    # Lógica para escrever o cabeçalho (nomes das colunas) no arquivo CSV apenas na primeira vez que o arquivo é criado.
    escrever_header = not os.path.exists(partial_results_filename)

    # Anexa (append) o resultado atual ao arquivo CSV de backup. 'mode='a'' significa 'append' (adicionar ao final).
    df_resultado_atual.to_csv(
        partial_results_filename,
        mode='a',
        header=escrever_header,
        index=False # Não salva o índice do DataFrame no arquivo.
    )
    
    # Incrementa o contador de itens processados nesta sessão.
    processed_in_this_session += 1

print("\nExtração de todos os novos links concluída.")
# Ao final de todo o processo, fecha a última instância do navegador para liberar os recursos do sistema.
driver.quit()

## 5. A Etapa Final: Unificando os Dados

Com a maratona de extração concluída, o robô agora tem dois arquivos importantes: o CSV original com as informações básicas e o CSV de resultados parciais com todos os novos detalhes.

A tarefa final é juntar esses dois arquivos. Usaremos a coluna `ID Imóvel` como a "chave" para combinar as informações, garantindo que os detalhes de quartos, camas e banheiros sejam adicionados à linha correta do anúncio correspondente. O resultado é um único arquivo CSV, completo e enriquecido.

In [ ]:
print("\nMapeando dados extraídos de volta para o DataFrame completo...")

# Verifica se o arquivo de resultados parciais foi realmente criado (ou seja, se o loop de extração rodou pelo menos uma vez).
if os.path.exists(partial_results_filename):
    # Carrega todos os detalhes que foram extraídos e salvos no arquivo de backup.
    df_detalhes = pd.read_csv(partial_results_filename)

    # Define os nomes das colunas que foram adicionadas pelo scraper.
    colunas_novas = ['Quartos', 'Camas', 'Banheiros', 'Horário de Check-in', 'Horário de Check-out']

    # Remove essas colunas do DataFrame original, caso elas já existam de uma execução anterior.
    # 'errors='ignore'' evita que o programa dê um erro caso as colunas não existam para serem removidas.
    df_airbnb_sem_detalhes = df_airbnb.drop(columns=colunas_novas, errors='ignore')

    # A mágica da unificação: usa a função 'merge' do pandas para juntar os dois DataFrames.
    # Funciona como um PROCV/VLOOKUP no Excel, usando o 'ID Imóvel' como a chave de correspondência.
    # 'how='left'' garante que todos os anúncios do arquivo original (df_airbnb_sem_detalhes) sejam mantidos no resultado final.
    df_final = pd.merge(df_airbnb_sem_detalhes, df_detalhes, on='ID Imóvel', how='left')

    try:
        # Tenta salvar o DataFrame final e unificado em um novo arquivo CSV. 'index=False' evita salvar o índice do DataFrame como uma coluna.
        df_final.to_csv(final_output_filename, index=False, encoding='utf-8-sig') # 'utf-8-sig' ajuda na compatibilidade com Excel.
        print(f"\nSUCESSO! DataFrame final com {len(df_final)} linhas salvo em '{final_output_filename}'")
    except Exception as e:
        # Captura possíveis erros durante o salvamento do arquivo (ex: falta de permissão).
        print(f"\nOcorreu um erro ao salvar o arquivo CSV final: {e}")
else:
    # Mensagem para o caso de nenhum novo item ter sido processado nesta execução.
    print("\nNenhum dado novo foi extraído. O arquivo final não foi gerado.")